In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
import scipy
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
path="D:\\CDS513\\Assignment 1\\Data"
    
os.chdir(path)

# Read csv data file
# Data without feature standardization
df = pd.read_csv('Data_final.csv')

In [4]:
df

,User-id,Game-title,Genre,personal-rating
0,5250,Alien Swarm,Shooter,3
1,975449,Alien Swarm,Shooter,5
2,1950243,Alien Swarm,Shooter,3
3,2259650,Alien Swarm,Shooter,2
4,2753525,Alien Swarm,Shooter,2
...,...,...,...,...
42534,287824954,Close Your Eyes,Puzzle,5
42535,293303005,All Is Dust,Role-playing (RPG),5
42536,300248088,Sigils of Elohim,Puzzle,5
42537,306839937,Secret Files Tunguska,Point-and-click,5


# Finding # of interaction

In [5]:
users_interactions_count_df = df.groupby(['User-id', 'Game-title']).size().groupby('User-id').size()
print('# users: %d' % len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5]\
                                    .reset_index()[['User-id']]


users_with_few_interactions_df = users_interactions_count_df[users_interactions_count_df < 3]\
                                    .reset_index()[['User-id']]

print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))
print('# users with less than 3 interactions: %d' % len(users_with_few_interactions_df))

# users: 8476
# users with at least 5 interactions: 1719
# users with less than 3 interactions: 5936


In [6]:
interactions_from_selected_users_df = df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'User-id',
               right_on = 'User-id')

interactions_from_few_selected_users_df = df.merge(users_with_few_interactions_df, 
               how = 'right',
               left_on = 'User-id',
               right_on = 'User-id')

print('# of interactions from all users: %d' % len(df))
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))
print('# of interactions from users with less than 3 interactions: %d' % len(interactions_from_few_selected_users_df))

# of interactions from all users: 42539
# of interactions from users with at least 5 interactions: 32809
# of interactions from users with less than 3 interactions: 6932


# Split Dataset for training and testing

In [7]:
interactions_train_df, interactions_test_df = train_test_split(interactions_from_selected_users_df,
                                   stratify=interactions_from_selected_users_df['User-id'],
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 26247
# interactions on Test set: 6562


In [8]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_from_selected_users_df.set_index('User-id')
interactions_train_indexed_df = interactions_train_df.set_index('User-id')
interactions_test_indexed_df = interactions_test_df.set_index('User-id')

In [9]:
def get_items_interacted(UserID, interactions_df):
    # Get the user's data and merge in the game information.
    interacted_items = interactions_df.loc[UserID]['Game-title']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [30]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:
    def get_not_interacted_items_sample(self, UserID, sample_size, seed=42):
        interacted_items = get_items_interacted(UserID, interactions_full_indexed_df)
        all_items = set(df['Game-title'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, UserID):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[UserID]
        if type(interacted_values_testset['Game-title']) == pd.Series:
            user_interacted_items_testset = set(interacted_values_testset['Game-title'])
        else:
            user_interacted_items_testset = set(interacted_values_testset['Game-title'])
        interacted_items_count_testset = len(user_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        user_recs_df = model.recommend_items(UserID, 
                                               items_to_ignore=get_items_interacted(UserID, 
                                                                                    interactions_train_indexed_df), 
                                                                                    topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in user_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(UserID, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS)

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = user_recs_df[user_recs_df['Game-title'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['Game-title'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        user_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return user_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, UserID in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            user_metrics = self.evaluate_model_for_user(model, UserID)  
            user_metrics['UserID'] = UserID
            people_metrics.append(user_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()

In [31]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_train_df.pivot(index='User-id', 
                                                          columns='Game-title', 
                                                          values='personal-rating').fillna(0)

users_items_pivot_matrix_df

Game-title,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,100% Orange Juice,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,12 Labours of Hercules III Girl Power,140,16bit Trader,3 Stars of Destiny,4 Elements,...,eden*,fault milestone one,iBomber Defense,iBomber Defense Pacific,ibb & obb,rFactor,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,theHunter,theHunter Primal
User-id,,,,,,,,,,,,,,,,,,,,,
76767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
229911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
298950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
554278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285869008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286926470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288274882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
users_items_pivot_matrix = users_items_pivot_matrix_df.values
users_items_pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[76767,
 86540,
 229911,
 298950,
 554278,
 561758,
 748719,
 975449,
 1129452,
 1612666]

In [34]:
users_items_pivot_sparse_matrix = csr_matrix(users_items_pivot_matrix)
users_items_pivot_sparse_matrix

<1719x2341 sparse matrix of type '<class 'numpy.float64'>'
	with 26247 stored elements in Compressed Sparse Row format>

In [35]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = NUMBER_OF_FACTORS_MF)

In [36]:
U.shape

(1719, 15)

In [37]:
Vt.shape

(15, 2341)

In [38]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [39]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 0.02353467,  0.00038916,  0.0023705 , ..., -0.04616758,
        -0.02682426, -0.00700004],
       [ 0.01333063, -0.0036258 ,  0.00229625, ...,  0.01730179,
        -0.01409164,  0.00124777],
       [ 0.00114152,  0.00371465, -0.00163404, ...,  0.00648264,
        -0.01104993, -0.00707625],
       ...,
       [-0.00065312,  0.00236217,  0.00155478, ..., -0.00100149,
         0.02260799,  0.00340469],
       [ 0.04717135, -0.00282968,  0.00230242, ...,  0.05200642,
         0.0152525 , -0.00147782],
       [ 0.002922  ,  0.01728084,  0.00464707, ..., -0.00530567,
         0.12741598,  0.00857444]])

In [40]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())

In [41]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,76767,86540,229911,298950,554278,561758,748719,975449,1129452,1612666,...,281535206,282464648,283872900,285386664,285860790,285869008,286926470,288274882,293042025,294620986
Game-title,,,,,,,,,,,,,,,,,,,,,
1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),0.311967,0.311373,0.310663,0.311627,0.310427,0.315972,0.307662,0.317806,0.309801,0.312289,...,0.309285,0.310169,0.310800,0.310362,0.310893,0.310069,0.310417,0.310558,0.313344,0.310767
10 Second Ninja,0.310619,0.310385,0.310813,0.310238,0.310554,0.310464,0.310515,0.310019,0.311159,0.309986,...,0.310402,0.311074,0.310661,0.310796,0.310543,0.310537,0.310644,0.310734,0.310432,0.311603
100% Orange Juice,0.310734,0.310730,0.310501,0.311221,0.310734,0.311057,0.310943,0.310562,0.310405,0.310718,...,0.310958,0.310590,0.310525,0.310613,0.310490,0.310669,0.310552,0.310687,0.310730,0.310867
12 Labours of Hercules,0.310679,0.311287,0.309453,0.308787,0.309095,0.313768,0.310777,0.312452,0.309119,0.310985,...,0.311074,0.311641,0.312449,0.311052,0.312556,0.310683,0.310613,0.310948,0.309378,0.310935
12 Labours of Hercules II The Cretan Bull,0.311085,0.309501,0.309962,0.310073,0.310119,0.315412,0.310630,0.310003,0.309813,0.311753,...,0.309093,0.311031,0.309608,0.310736,0.314469,0.310645,0.311190,0.310890,0.308604,0.309944
12 Labours of Hercules III Girl Power,0.310814,0.310840,0.309659,0.308470,0.309436,0.314244,0.310741,0.311831,0.309372,0.311000,...,0.310594,0.311442,0.311744,0.310956,0.312981,0.310674,0.310734,0.310915,0.309139,0.310655
140,0.310597,0.310575,0.310617,0.310442,0.310600,0.310542,0.310546,0.310475,0.310612,0.310643,...,0.310549,0.310609,0.310658,0.310601,0.310626,0.310587,0.310596,0.310590,0.310582,0.310609
16bit Trader,0.311399,0.309571,0.310498,0.306595,0.311049,0.315654,0.311449,0.309658,0.310223,0.311200,...,0.309732,0.310614,0.309940,0.310605,0.314358,0.310826,0.311014,0.310742,0.308416,0.309643
3 Stars of Destiny,0.310257,0.312829,0.310351,0.305794,0.309742,0.308869,0.310786,0.313612,0.310128,0.308751,...,0.313151,0.311128,0.314173,0.310909,0.307941,0.310645,0.309759,0.310577,0.311336,0.311677


In [42]:
len(cf_preds_df.columns)

1719

In [75]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['Game-title'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'Game-title', 
                                                          right_on = 'Game-title')[['recStrength', \
                                                                                    'Game-title', \
                                                                                    'Genre']]

        #return self.items_df
        return recommendations_df.drop_duplicates()
    
cf_recommender_model = CFRecommender(cf_preds_df, df)

In [44]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...


C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

...

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

1718 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.25552, 'recall@10': 0.3084}


C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
C:\Users\Mackhem\AppData\Local\Temp\ipykernel_8652\1162594667.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample

,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,UserID
51,7,14,62,0.112903,0.225806,62990992
145,18,19,49,0.367347,0.387755,138941587
31,11,18,36,0.305556,0.500000,48798067
302,6,10,36,0.166667,0.277778,11403772
132,21,23,35,0.600000,0.657143,49893565
30,10,20,35,0.285714,0.571429,24469287
440,12,14,32,0.375000,0.437500,116876958
304,8,11,30,0.266667,0.366667,47457723
63,8,10,29,0.275862,0.344828,11373749
56,17,20,28,0.607143,0.714286,10599862


In [45]:
def inspect_interactions(user_id, test_set=True):
    if test_set:
        interactions_df = interactions_test_indexed_df
    else:
        interactions_df = interactions_train_indexed_df
    return interactions_df.loc[user_id]

In [48]:
inspect_interactions(138941587, test_set=False).head(20)

,Game-title,Genre,personal-rating
User-id,,,
138941587,Spelunky,Platform,2
138941587,Lilly and Sasha Curse of the Immortals,Role-playing (RPG),3
138941587,Nux,Shooter,3
138941587,9 Clues The Secret of Serpent Creek,Point-and-click,5
138941587,Age of Empires II HD Edition,Real Time Strategy (RTS),2
138941587,SUPER DISTRO,Adventure,3
138941587,Bad Rats,Platform,3
138941587,Blaster Shooter GunGuy!,Adventure,5
138941587,Mitos.is The Game,Real Time Strategy (RTS),4


In [63]:
cf_recommender_model.recommend_items(138941587, topn=20, verbose=True)

,recStrength,Game-title,Genre
0,0.721477,The Elder Scrolls V Skyrim,Role-playing (RPG)
614,0.700609,Borderlands 2,Shooter
957,0.699200,Terraria,Platform
1327,0.678377,The Binding of Isaac,Shooter
1446,0.678270,The Binding of Isaac Rebirth,Shooter
1516,0.667091,AdVenture Capitalist,Simulator
1605,0.644868,Super Meat Boy,Platform
1660,0.636589,Audiosurf,Music
1699,0.630217,Trine,Platform
1742,0.614566,Nidhogg,Fighting


Evaluating Collaborative Filtering (SVD Matrix Factorization) model...


KeyError: 24632218